In [1]:
import os,sys
import numpy as np
import gdspy as gp
import gdstools as tools

# When work with mask, make sure you put the cell center at it's box's geometry center, or put it somewhere reasonable and easy to remember. 
# The location of an object is controlled by `wafer.add(gp.CellReference(<cell name>,(X,Y)))`
# Try to keep all geometry in the same layer in one sub-cell. Don't make the whole structure too messy.

In [2]:
inputmask = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified/dicealign_ref/HVeV_v3_Mask_mk16.gds'
print('Input file: %s'%inputmask)    

outdir = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified'
if not os.path.exists(outdir):
    os.makedirs(outdir)
print('Output dir: %s'%outdir)

Input file: /Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified/dicealign_ref/HVeV_v3_Mask_mk16.gds
Output dir: /Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified


In [3]:
# Create a lib and load the original mask
gdsii=gp.GdsLibrary()
gl=gdsii.read_gds(inputmask)

In [4]:
# mask base --> the basic layout of the 4-inch wafer
maskbase = gp.Cell('MASKBASE')
baselayer = 3

## outer frame
parts=['ul', 'ur', 'll', 'lr']

for partname in parts:
    # load the pre-saved points for drawing the polygon 
    maskbase_pts = np.genfromtxt('./polygonpts_maskbase_%s.txt'%partname)
    # remove the void for devices won't appear on our current mask
    unq, count = np.unique(maskbase_pts, axis=0, return_counts=True)
    repeated_groups=unq[count>1]
    for repeated_group in repeated_groups:
        repeated_idx = np.argwhere(np.all(maskbase_pts == repeated_group, axis=1))
        if repeated_idx.ravel().size==0:
            continue
        else:
            maskbase_pts = np.delete(maskbase_pts, range(repeated_idx.ravel()[0],repeated_idx.ravel()[1]), axis=0)
    # draw the polygon and add it to the mask
    maskbase_pg  = gp.Polygon(maskbase_pts, layer=baselayer)
    maskbase.add(maskbase_pg)
    
## dicing lines
lines = [
    [(-33000.00000,33000.00000),(33000.00000,33000.00000)],
    [(-44000.00000, 22000.00000),(44000.00000, 22000.00000)],
    [(-42700.00000, 0.00000),(-13000.00000, 0.00000)],
    [(-10600.00000, 0.00000),(-500.00000, 0.00000)],
    [(500.00000, 0.00000),(10600.00000, 0.00000)],
    [(42700.00000, 0.00000),(13000.00000, 0.00000)],
    [(-44000.00000, -22000.00000),(44000.00000,-22000.00000)],
    [(-33000.00000,-33000.00000),(33000.00000,-33000.00000)],
    [(-22000.00000,44000.00000),(-22000.00000,-44000.00000)],
    [(0.00000,43000.00000),(0.00000,13500.00000)],
    [(0.00000,10060.00000),(0.00000,500.00000)],
    [(0.00000,-500.00000),(0.00000,-10600.00000)],
    [(0.00000,-43000.00000),(0.00000,-13500.00000)],
    [(22000.00000,44000.00000),(22000.00000,-44000.00000)],
    [(-11000.00000,33000.00000),(-11000.00000,22000.00000)],
    [(11000.00000,33000.00000),(11000.00000,22000.00000)],
    [(-11000.00000,-44000.00000),(-11000.00000,-33000.00000)],
    [(11000.00000,-44000.00000),(11000.00000,-33000.00000)],
    [(-44000.00000,11000.00000),(-500.00000,11000.00000)],
    [(44000.00000,11000.00000),(500.00000,11000.00000)],
    [(-44000.00000,-11000.00000),(-500.00000,-11000.00000)],
    [(44000.00000,-11000.00000),(500.00000,-11000.00000)],
    # mod
    [(-22000.00000,-27500.00000),(22000.00000,-27500.00000)],
    [(-22000.00000,-5500.00000),(22000.00000,-5500.00000)],
    [(-22000.00000,16500.00000),(22000.00000,16500.00000)],
    [(-22000.00000,38500.00000),(22000.00000,38500.00000)],
    [(-44000.00000,5500.00000),(-22000.00000,5500.00000)],
    [(44000.00000,5500.00000),(22000.00000,5500.00000)],
    [(-44000.00000,-16500.00000),(-22000.00000,-16500.00000)],
    [(44000.00000,-16500.00000),(22000.00000,-16500.00000)],
    
    [(-11000.00000,11000.00000),(-11000.00000,0.00000)],
    [(11000.00000,11000.00000),(11000.00000,0.00000)],
    [(-11000.00000,-22000.00000),(-11000.00000,-11000.00000)],
    [(11000.00000,-22000.00000),(11000.00000,-11000.00000)],
    
    [(-33000.00000,-11000.00000),(-33000.00000,0.00000)],
    [(33000.00000,-11000.00000),(33000.00000,0.00000)],
    [(-33000.00000,22000.00000),(-33000.00000,11000.00000)],
    [(33000.00000,22000.00000),(33000.00000,11000.00000)],
]

for line in lines:
    maskbase_pp = gp.PolyPath(line, 4, layer=baselayer)
    maskbase.add(maskbase_pp)

/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_93054/1817406168.py:70: DeprecationWarning: [GDSPY] PolyPath is deprecated favor of FlexPath and will be removed in a future version of Gdspy.
  maskbase_pp = gp.PolyPath(line, 4, layer=baselayer)


In [5]:
# alignment marks
alignmarks=gl.extract('Alignment_Marks')
maskbase.add(alignmarks)

/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_93054/2818125711.py:2: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  alignmarks=gl.extract('Alignment_Marks')


In [6]:
# get kids
inputmask = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified/nukid_shrink10000.gds' 
gl=gdsii.read_gds(inputmask, rename_template='{name}_sq')
wafer_sq = gl.extract('WAFER_sq')

inputmask = '/Users/bicep/Documents/DeviceDesign/MKID_fab/CITdesign/SUmodified/slim_nuKid.gds' 
gl=gdsii.read_gds(inputmask, rename_template='{name}_slim')
wafer_slim = gl.extract('WAFER_slim')



/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_93054/3316403626.py:4: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  wafer_sq = gl.extract('WAFER_sq')
/var/folders/rn/l47h67f51194g8bjz60lqwlh0000gq/T/ipykernel_93054/3316403626.py:8: DeprecationWarning: [GDSPY] extract and the use of the global library is deprecated.
  wafer_slim = gl.extract('WAFER_slim')


In [7]:
# final mask
mask = gp.Cell('MASK')

mask.add(gp.CellReference(maskbase,(0,0)))

#NK=nukid, 2302=Feb2023, X=undefined date, CL/CQ=chip number slim/square
serlayer=2
ic = 0

# square
for x in range(-16500, 27500, 11000):
    for y in range(-38500, 49500, 22000):
        mask.add(gp.CellReference(wafer_sq,(x,y),x_reflection=(x>0),rotation=180*(x>0)))
        ic +=1
        text = gp.Text('NK2302XCQ%02d'%ic, 200, (x-5000, y-5000), layer=serlayer)
        mask.add(text)
        
for x in range(-27500, 82500, 55000):
    for y in range(-27500, 82500, 55000):
        mask.add(gp.CellReference(wafer_sq,(x,y),x_reflection=(x>0),rotation=180*(x>0)))
        ic +=1
        text = gp.Text('NK2302XCQ%02d'%ic, 200, (x-5000, y-5000), layer=serlayer)
        mask.add(text)
        
for x in range(-38500, 49500, 11000):
    for y in range(-5500, 38500, 22000):
        if abs(x)<27500:
            continue
        mask.add(gp.CellReference(wafer_sq,(x,y),x_reflection=(x>0),rotation=180*(x>0)))
        ic +=1
        text = gp.Text('NK2302XCQ%02d'%ic, 200, (x-5000, y-5000), layer=serlayer)
        mask.add(text)

ic = 0
for x in range(-11000, 33000, 22000):
    for y in range(-30250, 57750, 22000):
        mask.add(gp.CellReference(wafer_slim,(x,y),x_reflection=(x>0),rotation=180*(x>0)))
        ic +=1
        text = gp.Text('NK2302XCL%02d'%ic, 200, (x-10500, y-2250), layer=serlayer)
        mask.add(text)
        mask.add(gp.CellReference(wafer_slim,(x,y+5500),x_reflection=(x>0),rotation=180*(x>0)))
        ic +=1
        text = gp.Text('NK2302XCL%02d'%ic, 200, (x-10500, y+5500-2250), layer=serlayer)
        mask.add(text)
        
for x in [-33000, 33000]:
    for y in [-19250, 2750]:
        mask.add(gp.CellReference(wafer_slim,(x,y),x_reflection=(x>0),rotation=180*(x>0)))
        ic +=1
        text = gp.Text('NK2302XCL%02d'%ic, 200, (x-10500, y-2250), layer=serlayer)
        mask.add(text)
        mask.add(gp.CellReference(wafer_slim,(x,y+5500),x_reflection=(x>0),rotation=180*(x>0)))
        ic +=1
        text = gp.Text('NK2302XCL%02d'%ic, 200, (x-10500, y+5500-2250), layer=serlayer)
        mask.add(text)
        
gp.write_gds('%s/nukid_NK2302X.gds'%outdir, cells=sorted(tools.GetSubcellNames(mask)))